<a href="https://colab.research.google.com/github/Waxpple/COLAB_NEO4J/blob/master/Use_Neo4j_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install py2neo
from py2neo import Graph
#bolt://###[your-ip]:[your-port],auth=("user-account","password")
graph = Graph("",auth=())

In [19]:
"!pip install py2neo --upgrade

Requirement already up-to-date: py2neo in /usr/local/lib/python3.6/dist-packages (4.3.0)


In [34]:
graph.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")
for record in graph.run('''
LOAD CSV WITH HEADERS FROM "https://www.macalester.edu/~abeverid/data/stormofswords.csv" AS row
MERGE (src:Character {name: row.Source})
MERGE (tgt:Character {name: row.Target})
MERGE (src)-[r:INTERACTS]->(tgt)
SET r.weight = toInt(row.Weight)
RETURN count(*) AS paths_written
'''):
    print(record)

<Record paths_written=352>


In [0]:
for record in graph.run('''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book1-edges.csv" AS row
MATCH (src:Character {name: row.Source})
MATCH (tgt:Character {name: row.Target})
// relationship for the book
MERGE (src)-[r:INTERACTS1]->(tgt) ON CREATE SET r.weight = toInt(row.weight), r.book=1
'''):
    print(record)

In [0]:
graph.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")

for record in graph.run('''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book2-edges.csv" AS row
MATCH (src:Character {name: row.Source})
MATCH (tgt:Character {name: row.Target})
// relationship for the book
MERGE (src)-[r:INTERACTS2]->(tgt) ON CREATE SET r.weight = toInt(row.weight), r.book=2
'''):
    print(record)

In [0]:
graph.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")

for record in graph.run('''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book3-edges.csv" AS row
MATCH (src:Character {name: row.Source})
MATCH (tgt:Character {name: row.Target})
// relationship for the book
MERGE (src)-[r:INTERACTS3]->(tgt) ON CREATE SET r.weight = toInt(row.weight), r.book=3
'''):
    print(record)

In [0]:
graph.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")

for record in graph.run('''
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/mathbeveridge/asoiaf/master/data/asoiaf-book45-edges.csv" AS row
MATCH (src:Character {name: row.Source})
MATCH (tgt:Character {name: row.Target})
// relationship for the book
MERGE (src)-[r:INTERACTS45]->(tgt) ON CREATE SET r.weight = toInt(row.weight), r.book=45
'''):
    print(record)

In [39]:
for r in graph.run('''
MATCH p=(:Character)-[:INTERACTS]-(:Character)
RETURN p
'''):
    print(r)

<Record p=(Belwas)<-[:INTERACTS {weight: 13}]-(Jorah)>
<Record p=(Belwas)<-[:INTERACTS {weight: 26}]-(Daenerys)>
<Record p=(Belwas)-[:INTERACTS {weight: 10}]->(Illyrio)>
<Record p=(Belwas)-[:INTERACTS {weight: 18}]->(Barristan)>
<Record p=(Beric)<-[:INTERACTS {weight: 15}]-(Sandor)>
<Record p=(Beric)<-[:INTERACTS {weight: 23}]-(Arya)>
<Record p=(Beric)<-[:INTERACTS {weight: 8}]-(Eddard)>
<Record p=(Beric)-[:INTERACTS {weight: 21}]->(Thoros)>
<Record p=(Beric)-[:INTERACTS {weight: 4}]->(Gendry)>
<Record p=(Beric)-[:INTERACTS {weight: 4}]->(Anguy)>
<Record p=(Bran)<-[:INTERACTS {weight: 10}]-(Sansa)>
<Record p=(Bran)<-[:INTERACTS {weight: 9}]-(Arya)>
<Record p=(Bran)<-[:INTERACTS {weight: 23}]-(Robb)>
<Record p=(Bran)<-[:INTERACTS {weight: 15}]-(Eddard)>
<Record p=(Bran)<-[:INTERACTS {weight: 4}]-(Catelyn)>
<Record p=(Bran)-[:INTERACTS {weight: 11}]->(Samwell)>
<Record p=(Bran)-[:INTERACTS {weight: 11}]->(Theon)>
<Record p=(Bran)-[:INTERACTS {weight: 14}]->(Nan)>
<Record p=(Bran)-[:INTER

In [40]:
for record in graph.run("MATCH (c:Character) RETURN count(c) AS num"):
    print(record)

<Record num=107>


In [41]:
for record in graph.run('''
MATCH (c:Character)-[:INTERACTS]->()
WITH c, count(*) AS num
RETURN min(num) AS min, max(num) AS max, avg(num) AS avg_characters, stdev(num) AS stdev
'''):
    print(record)

<Record min=1 max=24 avg_characters=4.957746478873239 stdev=6.2276723918750845>


In [42]:
for r in graph.run('''
// Find maximum diameter of network
// maximum shortest path between two nodes
MATCH (a:Character), (b:Character) WHERE id(a) > id(b)
MATCH p=shortestPath((a)-[:INTERACTS*]-(b))
RETURN length(p) AS len, extract(x IN nodes(p) | x.name) AS path
ORDER BY len DESC LIMIT 4
'''):
    print(r)

<Record len=6 path=['Illyrio', 'Belwas', 'Daenerys', 'Robert', 'Sansa', 'Bran', 'Jojen']>
<Record len=6 path=['Illyrio', 'Belwas', 'Daenerys', 'Robert', 'Sansa', 'Bran', 'Luwin']>
<Record len=6 path=['Nan', 'Bran', 'Sansa', 'Robert', 'Daenerys', 'Belwas', 'Illyrio']>
<Record len=6 path=['Illyrio', 'Belwas', 'Daenerys', 'Robert', 'Stannis', 'Davos', 'Shireen']>


In [43]:
for r in graph.run('''
// Shortest path from Catelyn Stark to Khal Drogo
MATCH (catelyn:Character {name: "Catelyn"}), (drogo:Character {name: "Drogo"})
MATCH p=shortestPath((catelyn)-[INTERACTS*]-(drogo))
RETURN p
'''):
    print(r)

<Record p=(Catelyn)-[:INTERACTS {weight: 19}]->(Jaime)-[:INTERACTS {weight: 4}]->(Barristan)<-[:INTERACTS {weight: 11}]-(Jorah)-[:INTERACTS {weight: 6}]->(Drogo)>


In [44]:
for r in graph.run('''
MATCH (catelyn:Character {name: "Catelyn"}), (drogo:Character {name: "Drogo"})
MATCH p=allShortestPaths((catelyn)-[INTERACTS*]-(drogo))
RETURN p
'''):
    print(r)

<Record p=(Catelyn)-[:INTERACTS {weight: 8}]->(Sansa)-[:INTERACTS {weight: 5}]->(Robert)<-[:INTERACTS {weight: 5}]-(Daenerys)-[:INTERACTS {weight: 18}]->(Drogo)>
<Record p=(Catelyn)-[:INTERACTS {weight: 19}]->(Jaime)-[:INTERACTS {weight: 4}]->(Barristan)<-[:INTERACTS {weight: 11}]-(Jorah)-[:INTERACTS {weight: 6}]->(Drogo)>
<Record p=(Catelyn)-[:INTERACTS {weight: 19}]->(Jaime)-[:INTERACTS {weight: 4}]->(Barristan)<-[:INTERACTS {weight: 20}]-(Daenerys)-[:INTERACTS {weight: 18}]->(Drogo)>
<Record p=(Catelyn)-[:INTERACTS {weight: 19}]->(Jaime)-[:INTERACTS {weight: 17}]->(Robert)<-[:INTERACTS {weight: 5}]-(Daenerys)-[:INTERACTS {weight: 18}]->(Drogo)>
<Record p=(Catelyn)-[:INTERACTS {weight: 4}]->(Stannis)<-[:INTERACTS {weight: 5}]-(Robert)<-[:INTERACTS {weight: 5}]-(Daenerys)-[:INTERACTS {weight: 18}]->(Drogo)>
<Record p=(Catelyn)-[:INTERACTS {weight: 4}]->(Cersei)-[:INTERACTS {weight: 16}]->(Robert)<-[:INTERACTS {weight: 5}]-(Daenerys)-[:INTERACTS {weight: 18}]->(Drogo)>
<Record p=(Catel

In [46]:
for r in graph.run('''
// Find all pivotal nodes in network
MATCH (a:Character), (b:Character) WHERE id(a) > id(b)
MATCH p=allShortestPaths((a)-[:INTERACTS*]-(b)) WITH collect(p) AS paths, a, b
UNWIND nodes(head(paths)) as c // first path
WITH * WHERE NOT c IN [a,b] AND all(path IN tail(paths) WHERE c IN nodes(path))
RETURN a.name, b.name, c.name AS PivotalNode, length(head(paths)) as pathLength, length(paths) as pathCount SKIP 490 LIMIT 10
'''):
    print(r)

<Record a.name='Aerys' b.name='Doran' PivotalNode='Tyrion' pathLength=2 pathCount=1>
<Record a.name='Meera' b.name='Jorah' PivotalNode='Jon' pathLength=4 pathCount=3>
<Record a.name='Meera' b.name='Jorah' PivotalNode='Robert' pathLength=4 pathCount=3>
<Record a.name='Marillion' b.name='Bran' PivotalNode='Sansa' pathLength=2 pathCount=1>
<Record a.name='Olenna' b.name='Mance' PivotalNode='Sansa' pathLength=3 pathCount=1>
<Record a.name='Olenna' b.name='Mance' PivotalNode='Jon' pathLength=3 pathCount=1>
<Record a.name='Lancel' b.name='Ygritte' PivotalNode='Kevan' pathLength=4 pathCount=1>
<Record a.name='Lancel' b.name='Ygritte' PivotalNode='Sansa' pathLength=4 pathCount=1>
<Record a.name='Lancel' b.name='Ygritte' PivotalNode='Jon' pathLength=4 pathCount=1>
<Record a.name='Cressen' b.name='Bran' PivotalNode='Davos' pathLength=4 pathCount=6>


In [47]:
for r in graph.run('''
MATCH (c:Character)-[:INTERACTS]-()
RETURN c.name AS character, count(*) AS degree ORDER BY degree DESC LIMIT 10
'''):
    print(r)

<Record character='Tyrion' degree=36>
<Record character='Jon' degree=26>
<Record character='Sansa' degree=26>
<Record character='Robb' degree=25>
<Record character='Jaime' degree=24>
<Record character='Tywin' degree=22>
<Record character='Cersei' degree=20>
<Record character='Arya' degree=19>
<Record character='Robert' degree=18>
<Record character='Joffrey' degree=18>


In [48]:
for r in graph.run('''
MATCH (c:Character)-[r:INTERACTS]-()
RETURN c.name AS character, sum(r.weight) AS weightedDegree ORDER BY weightedDegree DESC LIMIT 10
'''):
    print(r)

<Record character='Tyrion' weightedDegree=551>
<Record character='Jon' weightedDegree=442>
<Record character='Sansa' weightedDegree=383>
<Record character='Jaime' weightedDegree=372>
<Record character='Bran' weightedDegree=344>
<Record character='Robb' weightedDegree=342>
<Record character='Samwell' weightedDegree=282>
<Record character='Arya' weightedDegree=269>
<Record character='Joffrey' weightedDegree=255>
<Record character='Daenerys' weightedDegree=232>


In [49]:
for r in graph.run('''
MATCH (c:Character)
WITH collect(c) AS characters
CALL apoc.algo.betweenness(['INTERACTS'], characters, 'BOTH') YIELD node, score
SET node.betweenness = score
RETURN node.name AS name, score ORDER BY score DESC LIMIT 10
'''):
    print(r)

<Record name='Jon' score=1279.7533534055324>
<Record name='Robert' score=1165.6025171231624>
<Record name='Tyrion' score=1101.3849724234349>
<Record name='Daenerys' score=874.8372110508582>
<Record name='Robb' score=706.5572832464791>
<Record name='Sansa' score=705.1985623519137>
<Record name='Stannis' score=571.5247305125714>
<Record name='Jaime' score=556.1852522889822>
<Record name='Arya' score=443.0135843004335>
<Record name='Tywin' score=364.72121955280846>


In [50]:
for r in graph.run('''
MATCH (c:Character)
WITH collect(c) AS characters
CALL apoc.algo.closeness(['INTERACTS'], characters, 'BOTH') YIELD node, score
RETURN node.name AS name, score ORDER BY score DESC LIMIT 10
'''):
    print(r)

<Record name='Tyrion' score=0.004830917874396135>
<Record name='Sansa' score=0.004807692307692308>
<Record name='Robert' score=0.0047169811320754715>
<Record name='Robb' score=0.004608294930875576>
<Record name='Arya' score=0.0045871559633027525>
<Record name='Jon' score=0.004524886877828055>
<Record name='Stannis' score=0.004524886877828055>
<Record name='Jaime' score=0.004524886877828055>
<Record name='Tywin' score=0.004424778761061947>
<Record name='Eddard' score=0.004347826086956522>


In [51]:
!pip install python-igraph
from igraph import Graph as IGraph

query = '''
MATCH (c1:Character)-[r:INTERACTS]->(c2:Character)
RETURN c1.name, c2.name, r.weight AS weight
'''

ig = IGraph.TupleList(graph.run(query), weights=True)

ig

     |████████████████████████████████| 378kB 2.8MB/s 
  Created wheel for python-igraph: filename=python_igraph-0.7.1.post6-cp36-cp36m-linux_x86_64.whl size=2218389 sha256=e7cbc0d340a0cd8c934f6d1dc332b99a41d4538feb1844dcd74f154e335e1642
  Stored in directory: /root/.cache/pip/wheels/41/d6/02/34eebae97e25f5b87d60f4c0687e00523e3f244fa41bc3f4a7
Successfully built python-igraph
